In [1]:
# import libraries
import pandas as pd
import camelot

## Get tables from pdf file


In [9]:
url_file = "https://camelot-py.readthedocs.io/en/master/_static/pdf/foo.pdf"


def get_table_from_pdf(
    path_file: str = url_file, method="lattice"):
    # Get the tables in the PDF file
    if path_file.endswith(".pdf"):
        try:
            tables = camelot.read_pdf(filepath=path_file, pages="all", flavor=method)
            tables.export(r'foo.csv', f='csv', compress=True)
            print(f"{path_file} successfully loaded!")
            if len(tables) > 0:
                print(f"{len(tables)} tables extracted")
                return tables
            else:
                raise Exception("No table extracted!")
        except Exception as e:
            print(e)
            print("Try change -method- parameter to 'stream'!")
            # tables = camelot.read_pdf(filepath=url_file, pages="all", flavor="lattice")
    else:
        raise Exception("File is not a PDF")


In [10]:
# set path to pdf file
pdf_file = r"../files/Rapport potablité Eau RADESS 21-01-24.pdf"

tables = get_table_from_pdf(path_file=pdf_file, method="stream")


../files/Rapport potablité Eau RADESS 21-01-24.pdf successfully loaded!
11 tables extracted


## Insight Extracted Tables


In [11]:
def display_tables_info(tables) -> pd.DataFrame:
    assert len(tables) > 0, "No tables Found!"
    table_infos = [
        table.parsing_report | {"n_rows": table.shape[0], "n_cols": table.shape[1]}
        for table in tables
    ]
    return pd.DataFrame(table_infos)


display(display_tables_info(tables))


,accuracy,whitespace,order,page,n_rows,n_cols
0,95.42,58.39,1,1,23,7
1,97.62,43.45,2,1,21,8
2,99.61,29.62,1,2,46,8
3,99.50,28.57,1,3,35,8
4,94.93,37.50,2,3,16,2
5,99.72,27.29,1,4,57,9
6,99.84,12.73,1,5,48,9
7,99.85,5.05,1,6,44,9
8,99.84,8.94,1,7,46,9
9,99.74,4.94,1,8,45,9


## View a table as Data Frame


In [12]:
# print the first table as Pandas DataFrame
df_table = tables[0].df
display(df_table)


,0,1,2,3,4,5,6
0,Code échantillon : 566-01/01,,Référence du client : RADEES 728,,,Date/heure début d'analyse : 22/01/2024 à 10h20,
1,Lieu d’exécution des analyses : LC2A,,"Condition de réception : T°C : 6,3°C",,Date d’édition : 28/01/2024,,
2,Référence de la méthode d’échantillonnage:,Volume : 6 litre,,,Conditions spécifiques : 3°C à 8°C,,
3,INSPC/15/V01,,,,,,
4,,,,,,Critères,
5,,,,,Incertitude,,
6,Paramètre(s) microbiologiques,Méthode/Version,Résultat,(unité,,microbiologiques,Appréciation
7,,,,,(%),,
8,,,,,,Marocains(1) (VMA),
9,Dénombrement de micro-organismes,,,,,,


## Export tables in csv format into zip file


In [5]:
tables.export(
    "files/data.csv", f="csv", compress=True
)  # json, excel, html, markdown, sqlite
